In [40]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService


import time
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import requests
import pprint
max_chrome_driver_path = '/Users/maxlien/Downloads/chromedriver-mac-arm64/chromedriver'

from dotenv import load_dotenv
import os

def config():
    load_dotenv()
  
config()

username = str(os.getenv('MONOPLE_GMAIL'))
password=str(os.getenv('MONOPLE_PASSWORD'))

In [41]:
url = 'https://math.stackexchange.com/questions/3665900/determining-whether-a-matrix-is-positive-definite-from-its-lu-decomposition'
res = requests.get(url)
soup = BeautifulSoup(res.text, parser='html_parser')
post_body = soup.find_all('div', class_='s-prose js-post-body',itemprop='text')
question_title = soup.find('h1',class_='fs-headline1 ow-break-word mb8 flex--item fl1',itemprop='name')

answer_body = [i.text for i in post_body if i.text]
answer_body

['\nGiven that $A=LU$ where $L$ and $U$ are (known) lower and upper triangular matrices, is there any simple way to determine whether $A$ is positive definite?\nBackground\nI have been using this algorithm on Wikipedia to compute LU decompositions.\nThe comment section says:\n\nINPUT: A ... a square matrix having dimension N\nOUTPUT: Matrix A is changed, it contains a copy of both matrices L-E and U as A=(L-E)+U such that PA=LU.\n\nI\'m not sure what exactly \'E\' refers to here, but I understand that \'L\' refers to a lower triangular matrix, and \'U\' to an upper triangular matrix, and I understand that the algorithm is letting these two cohabit the same square matrix.\nI seek to determine whether the original matrix $A$ is positive definite. I have a feeling that $A$ is positive definite only if $A$ is symmetric and (after running the algorithm) all of the diagonal elements are positive. Is this correct? (Or is there some other way to determine whether $A$ is positive definite, from

In [42]:
answer_body[1:len(answer_body)]

['\nWe assume that $A\\in M_n$ is a real matrix.\nThere are  $2$ definitions for $A>0$.\ni)  $A$ is symmetric and, for every $x\\not= 0$, $x^TAx>0$.\nii) For every $x\\not= 0$, $x^TAx>0$.\nThen ii) is equivalent to i) for the symmetric matrix $A+A^T$.\nThus we may assume that $A$ is symmetric.\nThen the fastest method to see if $A>0$ is to use the standard algorithm that calculates the Choleski deomposition of $A$.\n$\\bullet$ if the algo. runs completely, then $A>0$.\n$\\bullet$ If the algo. stops with an error message, then $A$ is not $>0$.\n',
 '\nWe can determine whether $A>0$ by examining the diagonals of $L$ and $U$.\n$$A>0 \\iff l_{i,i}u_{i,i}>0 \\forall i$$\nThese products $l_{i,i}u_{i,i}$ are the diagonal elements of $D$ in the LDU decomposition, and $A>0\\iff D>0$ as shown here.\n(In the algorithm referenced from Wikipedia it is simply a matter of testing whether all diagonal elements in the output are positive.)\n']

In [43]:
a =[]
for i in content:
    if i.text:
        a.append(i.text)
len(a)

3

In [46]:
class StackExchangeScraper:
    def __init__(self, chrome_driver_path=max_chrome_driver_path,username=username, password=password):
        self.driver_path = chrome_driver_path
        self.options = webdriver.ChromeOptions()
        #print(self.options)
        self.service = ChromeService(executable_path=self.driver_path)
        self.driver = webdriver.Chrome(service=self.service) #executable_path= max_chrome_driver_path)#,options=self.options)#executable_path=self.driver_path)
        self.wait = WebDriverWait(self.driver, 10)
        
        self.username =username
        self.password = password

    # ... rest of the class methods remain unchanged ...

# Note: Only the initialization method of the class is shown here for brevity.
# The rest of the class methods would remain unchanged from the previous version.
    def _set_up_agent(self):
        #set useragent
        self.options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    # def _setup_headless_driver(self):
    #     """
    #     Sets up a headless Selenium Chrome driver.
    #     """
    #     self.options.add_argument('--headless')
    #     self.options.add_argument('--no-sandbox')
    #     self.options.add_argument('--disable-dev-shm-usage')
    #     self.driver = webdriver.Chrome(self.driver_path, options=options)
    def _wait(self,type, name):
        return self.wait.until(
                EC.presence_of_element_located((type, name))
            )
    def _wait_all(self, type, name):
        return self.wait.until(
                EC.visibility_of_all_elements_located((type, name))
            )
    def _login(self, username, password):
        
        
        login_button_XPATH = '/html/body/div[1]/header[1]/div/nav/ol/li[4]/a'
        login_button = self._wait(By.XPATH, login_button_XPATH)
        self.driver.execute_script("arguments[0].scrollIntoView();", login_button)
        login_button.click()


        
    

        username_box = self._wait(By.XPATH, '//*[@id="email"]')
        username_box.send_keys(username)

        password_box = self._wait(By.XPATH,'//*[@id="password"]' )
        password_box.send_keys(password)


        submit_button =self._wait(By.XPATH, '//*[@id="submit-button"]') 
        self.driver.execute_script("arguments[0].scrollIntoView();", submit_button)
        submit_button.click()




    def search_links(self, input):
        self._set_up_agent()
        self.driver.get('https://stackexchange.com')

        self._login(self.username, self.password)

        search_box = self._wait(By.XPATH, '//*[@id="search"]/div/input')
        search_box.send_keys(input)
        search_box.send_keys(Keys.ENTER)


        content = self._wait(By.XPATH, '//*[@id="content"]/div/div[3]')
        html_content = content.get_attribute('outerHTML')
        soup = BeautifulSoup(html_content, 'html.parser')
        links = soup.find_all(class_='result-link')
        # for link in links:
        #     print(link.a['href'])
        urls = [str(link.a['href']) for link in links]
        return urls       
        # try:
        #     #for stack_exchange
        #     #result_link_elements = self._wait_all(By.CSS_SELECTOR, "result-link > a")
        #     #print(result_link_elements)
        #     # Extracting the links within each of the 'result-link' elements
        #     links =[]
        #     for element in self._wait_all(By.CSS_SELECTOR, "result-link > a"):
        #         links.append(element.get_attribute("herf"))
        # except TimeoutException :
        #     #in case it redirect to meta.stackexchange
        #     #result_link_elements = self._wait_all(By.CLASS_NAME, "s-post-summary--content-title > a")
        #     #print(result_link_elements)
        #     # Extracting the links within each of the 'result-link' elements
        #     links = []
        #     for element in self._wait_all(By.CSS_SELECTOR, "s-post-summary--content-title > a"):
        #         links.append(element.get_attribute("innerHTML"))

        
        # return links


    def search_anwser(self,url):
        content = dict()
        answer_body = []


        res = requests.get(str(url))
        soup = BeautifulSoup(res.text, parser='html_parser')
        post_body = soup.find_all('div', class_='s-prose js-post-body',itemprop='text')
        question_title = soup.find('h1',class_='fs-headline1 ow-break-word mb8 flex--item fl1',itemprop='name')

        answer_body = [i.text for i in post_body if i.text]


        content['question_title'] = question_title
        content['question'] = answer_body[0]
        content['answer'] = answer_body[1:len(content)]

        return content
        



    def scrape(self, input):
        """
        Main method to scrape StackExchange for a given query and return the texts in a list.
        """
        #self._setup_headless_driver()
        
        urls = self.search_links(input)
        response = {}
        for url in urls:
            content = self.search_anwser(url)
            response[f'{url}'] = content

        self.driver.quit()

        return response

# Note: As before, the Selenium portion cannot be executed in this environment. 
# The code should be run in a local environment with the appropriate browser driver.


In [47]:
SS = StackExchangeScraper(chrome_driver_path=max_chrome_driver_path,username=username, password=password)
content =SS.scrape('what is positive matter')
content

{'https://stackoverflow.com/questions/55769540/position-sticky-not-working-no-matter-what': {'question_title': <h1 class="fs-headline1 ow-break-word mb8 flex--item fl1" itemprop="name"><a class="question-hyperlink" href="/questions/55769540/position-sticky-not-working-no-matter-what">Position: sticky; not working no matter what</a></h1>,
  'question': '\nTrying to get a class to remain sticky and constantly be in view when the user scrolls down the page, but doesn\'t seem to work no matter what. \nI tired: \n.entry-summary{\nposition: -webkit-sticky; /* Safari */\n  position: sticky;\n  top: 0;}\n\nBut this does nothing. \nThe full code can be seen below and is part of this webpage https://aqwaa.net/product/constriction/: \n<div class="qode-single-product-summary">\n    <div class="summary entry-summary">\n        <div class="clearfix">\n            <h1 class="product_title entry-title">Constriction</h1><p class="price"><span class="woocs_price_code" data-product-id="459"><span class="

In [50]:
soup = BeautifulSoup(content, parser='html.parser')
links = soup.find_all(class_='result-link')
for link in links:
    print(link.a['href'])
a_links = [link.a['href'] for link in a]
# print(len(links))


https://stackoverflow.com/questions/55769540/position-sticky-not-working-no-matter-what
https://math.stackexchange.com/questions/3665900/determining-whether-a-matrix-is-positive-definite-from-its-lu-decomposition
https://electronics.stackexchange.com/questions/499491/splicing-ac-adaptor-cord-which-is-positive-and-which-is-negative
https://electronics.stackexchange.com/questions/227190/why-doesnt-positive-negative-terminal-matter-in-some-electrical-applicances
https://stackoverflow.com/questions/7129597/how-to-keep-track-of-a-position-no-matter-what-screen-size-is
https://stackoverflow.com/questions/74447652/if-the-value-in-sql-is-positive-how-can-i-do-this-operation-if-it-is-negative
https://stackoverflow.com/questions/69641323/why-codeblocks-asking-me-to-add-a-after-elses-condition-if-i-add-a-th
https://worldbuilding.stackexchange.com/questions/101581/negative-energy-beam-negative-matter-projectiles
https://mathematica.stackexchange.com/questions/28659/manipulate-doesnt-work-for-plott

In [ ]:
driver.get('https://stackoverflow.com/questions/55769540/position-sticky-not-working-no-matter-what')


In [120]:

soup = BeautifulSoup(content, parser='html.parser')
#links = content.text
links = soup.find_all('a')
herfs = []
for link in links:
    herf = link.get('herf')
    herfs.append(herf)

TypeError: object of type 'WebElement' has no len()

In [ ]:
herfs

In [71]:
soup = BeautifulSoup(parser='html.parser', content=result)
soup.find_all('herf')

TypeError: __init__() got an unexpected keyword argument 'content'

In [67]:
dir(result[0])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_elements',
 'get_attribute',
 'get_dom_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'shadow_root',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']